In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
%tensorflow_version 2.x



In [ ]:
!pip uninstall keras -y
!pip uninstall keras-nightly -y
!pip uninstall keras-Preprocessing -y
!pip uninstall keras-vis -y
!pip uninstall tensorflow -y
!pip install napari[all]
!pip install tensorflow==2.2.0
!pip install keras==2.3.0
!pip install vollseg 
!pip install n2v


In [ ]:
%cd '/content/drive/My Drive/VollSeg/'
import os
import glob
import sys
import numpy as np
from tqdm import tqdm
from tifffile import imread, imwrite
from n2v.models import N2V
from csbdeep.models import Config, CARE
from vollseg import StarDist3D, UNET, VollSeg3D
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
from pathlib import Path
from n2v.models import N2V

In [ ]:
ImageDir = '/content/drive/My Drive/data/Raw/'
Model_Dir = '/content/drive/My Drive/data/'

SaveDir = ImageDir + 'Results/'

NoiseModelName = 'ScipyDenoising'
UNETModelName = 'UNETVolumeSeg'
StarModelName = 'VolumeSeg'
UnetModel = UNET(config = None, name = UNETModelName, basedir = Model_Dir)
StarModel = StarDist3D(config = None, name = StarModelName, basedir = Model_Dir)
NoiseModel = N2V(config=None, name=NoiseModelName, basedir=Model_Dir)

In [ ]:
Raw_path = os.path.join(ImageDir, '*.tif')
filesRaw = glob.glob(Raw_path)
filesRaw.sort
min_size = 1
n_tiles = (2,2,2)
dounet = False
for fname in filesRaw:
     
     image = imread(fname)
     Name = os.path.basename(os.path.splitext(fname)[0])
     VollSeg3D( image, unet_model, star_model, noise_model = noise_model, min_size = min_size,  n_tiles = n_tiles, UseProbability = True, save_dir = save_dir, Name = Name, dounet = dounet)
